# Pip

In [1]:

!pip install simplet5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 6.5 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 KB 33.8 MB/s eta 0:00:00
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6873 sha256=02df083d17cb6c5ad2d5bdec39c6f0ca737d831424a2c349809ca571f8a37f8b
  Stored in directory: /root/.cache/pip/wheels/30/1b/96/47ad5fda68def6635961fd0ca312fc0ce3670936ae09d97c43
Successfully built simplet5
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
  Attempting uninstall: pyDeprecate
    Found existing installation: pyDeprecate 0.3.2
    Uninstalling pyDeprecate-0.3.2:
      Successfully uninstalled pyDeprecate-0.3.2
  Attempting uninstall: transformers
    Found existing installation: transf

In [2]:
#also install streamlit
!pip install -q streamlit

# Knowledge you should have a grasp for understanding the code from here 


https://discuss.streamlit.io/t/avoid-rerunning-some-code/1313/4 - Helps you understand the execution flow 

https://docs.streamlit.io/library/api-reference/performance/st.cache - streamlit cache 

https://github.com/Shivanandroy/simpleT5 - simplet5 package 

https://docs.streamlit.io/library/api-reference/widgets/st.download_button - creating download button 

https://ngrok.com/ - ngrok for tunelling 

https://docs.streamlit.io/library/api-reference/widgets/st.text_input - input text 

and also the training and inference script!



# Create app.py file

In [16]:
%%writefile app.py 
#import
import streamlit as st
from simplet5 import SimpleT5
import random
import string
import numpy as np
from transformers import pipeline

#this will help us create unique keys for each element

def specific_string(length):  
     
    letters = string.ascii_lowercase # define the lower case string  
     # define the condition for random.choice() method  
    result = ''.join((random.choice(letters)) for x in range(length))  
    print(" Random generated string with repetition: ", result)  

#each question is written as "number) question" this command removes that number) part
def extract_main_thing(para,sep=" "):
    return " ".join(para.split(sep)[1:])
#We keep the model in cache otherwise according to streamlits functioning the model will be reloaded every time the users clicks a button the hash keys are necessary for encoding the different data type
#Quick note i have my model in ../input/training-t5-model-yet-again
@st.cache(hash_funcs={"simplet5.simplet5.SimpleT5":id,"transformers.pipelines.question_answering.QuestionAnsweringPipeline":id})
def load_models():
    model2 = SimpleT5()
     

    model2.load_model("t5","../input/training-t5-model-yet-again/outputs/simplet5-epoch-1-train-loss-1.7323-val-loss-1.8841",use_gpu=True)
    
    nlp = pipeline("question-answering")
    return model2,nlp
model2,nlp=load_models() 
#You can say they are like state objects we keep the previous question,slider,stride-percent nicely on memory before a new button happens to be clicked
@st.cache(allow_output_mutation=True)
def saving_temporary_questions():
    return {"questions":[],
            "answers":[],
           "prev":"",
           "slider":0,
           "stride-amount":70}
qa=saving_temporary_questions() 
#Creating Our Model 
class SlidingWindow():
    def __init__(self,words,stride,window):
        self.idx1=0 
        self.idx2=window 
        self.stride=stride
        self.words=words
    def get_item(self):
        if self.idx2-len(self.words)>=self.stride:
            return "END"
        output=self.words[self.idx1:min([len(self.words),self.idx2])] 
        self.idx1+=self.stride 
        self.idx2+=self.stride  
        return output 
def create_questions(text,number,stride_percent=70):
    words=text.split()  
    window_size=int(len(words)/number)
    stride=max([2,int(window_size*stride_percent/100)])
    slided=SlidingWindow(words,stride,window_size) 
    questions=[]
    answers=[]
    while True: 
        item=slided.get_item()
        if item=="END":
            break 
        questions+=model2.predict(" ".join(item))
        answers.append(nlp(question=questions[-1],context=text)['answer'])
    return questions,answers
        
#creating the layout        
st.title("AI powered question answer generator") 
input_text=st.text_area("Input your passage :","your passage to generate questions from here") 
no = st.slider('number of questions you want too many might harm performance', 0, 0,50)
stri=st.slider("stride percentage",0,0,100) 
#first we create a button called generate questions 
if st.checkbox("generate questions"): 
    st.text("working on it") 
    #we need to check if any changes have been observed in the key parameters then only we should regenrate the questions 
    #the person might just happened to click another button in that case we dont want this part to rexecute
    question_elements=[] 
    answer_elements=[] 
    if qa["prev"]!=input_text or qa["slider"]!=int(no) or qa["stride-amount"]!=int(stri):
        qa["questions"],qa["answers"]=create_questions(input_text,no,int(stri))
        qa["prev"]=str(input_text)
        qa["slider"]=int(no)
        qa["stride-amount"]=int(stri)
    #creating two columns one with delete option another q and a 
    column1,column2=st.columns(2)
    mask=list([1 for _ in qa["questions"]]) 
    
    with column1:
            st.text("\n \n")
            i=0
            for question,answer in zip(qa["questions"],qa["answers"]):#loop over the questions and answers
                quee=st.text_input("question:-",value=f"{i}) {question}",key=str(i)+"q")
                anse=st.text_input("answer:-",value=f"answer {answer}",key=str(i)+"a")
                i+=1
                #store them to record text editing
                question_elements.append(quee)
                answer_elements.append(anse)
    with column2:
            checks=[] 
            #creating our checks which help deleting questions from the .txt output file
            st.text("Check a question to delete it")
            for i,_ in enumerate(qa["questions"]):
                checks.append(st.checkbox(f"~ {i}",key=str(i)))
                for __ in range(10):st.text("") 
    for index,element in enumerate(question_elements):#we see if any sort of question has been edited and replace the orginal with the edited text
       
        if f"{qa['questions'][index]}" not in str(element):
            qa["questions"][index]=extract_main_thing(str(element))
     
    for index,element in enumerate(answer_elements):#same for answers
        if str(element)!=f"answer {qa['answers'][index]}":
            qa["answers"][index]=extract_main_thing(str(element),"answer")
    #every time a question gets deleted we filter it in the mask unchecking it will reverse the process
    for index,c in enumerate(checks):
        if c:
            mask[index]=0 
    #create two buttons for respective purpose   
    d1=st.checkbox("Download with answers")
    d2=st.checkbox("Download only questions") 
    lines=[]
    if d1: 
        answers_m,questions_m=[],[]
        for i in range(len(mask)):
            if mask[i]==1:#we see if the mask is allowing the question
                answers_m.append(qa["answers"][i])
                questions_m.append(qa["questions"][i])
        #creating our output file
        for question,answer in zip(questions_m,answers_m):
            lines.append("question :- {}".format(question))
            lines.append("answer :- {}".format(answer))
        document=("\n".join(lines)).encode("utf-8")#encode it in utf-8 to create a download button
        st.download_button("click here to download your file [answers included]!",data=document,file_name="q and a.txt")#create the download button
    #same thing but only questions are included
    if d2: 
        questions_m=[]
        for i in range(len(mask)):
            if mask[i]==1: 
          
                questions_m.append(qa["questions"][i])
            
        for question in questions_m:
            lines.append("question :- {}".format(question))
       
        document=("\n".join(lines)).encode("utf-8")
        st.download_button("click here to download your file [answers excluded]!",data=document,file_name="q and a.txt")

Overwriting app.py


# downloading some ngrok packages to help us view the website [ u might not need to if u are on local pc]

In [4]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2022-05-23 05:25:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  13.4MB/s    in 1.0s    

2022-05-23 05:25:30 (13.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [5]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


# in the key section place ur auth token from ngrok u can get it from the following url :- 
https://dashboard.ngrok.com/get-started/your-authtoken

In [6]:
!./ngrok authtoken <key>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [13]:


get_ipython().system_raw('./ngrok http 8501 &')

# Tunelling the localhost address

In [14]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://ce48-34-77-112-124.ngrok.io


# Executing the app

In [ ]:
!streamlit run app.py


2022-05-23 05:57:50.662 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.19.2.2:8501
  External URL: http://34.77.112.124:8501



# Thank you!